In [3]:
import autogen
import langchain
import llama_index
import openai
import requests
import xml.etree.ElementTree as ET
from scholarly import scholarly

print("✅ All libraries imported successfully!")



✅ All libraries imported successfully!


In [7]:
def fetch_google_scholar_papers(query):
        """
            Fetches top 6 research papers from Google Scholar.
            Returns:
                list: A list of dictionaries containing paper details (title, summary, link)
        """
        papers = []
        search_results = scholarly.search_pubs(query)

        for i, paper in enumerate(search_results):
            if i >= 6:
                break
            papers.append({
                "title": paper["bib"]["title"],
                "summary": paper["bib"].get("abstract", "No summary available"),
                "link": paper.get("pub_url", "No link available")
            })
        return papers



In [8]:
res=fetch_google_scholar_papers("Multi-Agent LLM Systems")
for i, paper in enumerate(res):
    print(f"\n📄 Paper {i+1}: {paper['title']}")
    print(f"📝 Summary: {paper['summary']}")
    print(f"🔗 Link: {paper['link']}")



📄 Paper 1: A survey on LLM-based multi-agent systems: workflow, infrastructure, and challenges
📝 Summary: For system  LLM-based multi-agent systems in two main areas: problem-solving and world  simulation. Finally, we address several challenges faced by LLM-based multi-agent systems
🔗 Link: https://link.springer.com/article/10.1007/s44336-024-00009-2

📄 Paper 2: LLM multi-agent systems: Challenges and open problems
📝 Summary: multi-agent systems. We also explore the potential application of multi-agent systems in  blockchain systems to  their future development and application in real-world distributed systems.
🔗 Link: https://arxiv.org/abs/2402.03578

📄 Paper 3: Multi-agent collaboration: Harnessing the power of intelligent llm agents
📝 Summary: In fact, our proposed multi-agent system also aims to make strides toward achieving a higher  level of artificial general intelligence (AGI). The development and deployment of advanced
🔗 Link: https://arxiv.org/abs/2306.03314

📄 Paper 4: Llm-